In [1]:
import re
import openai
import json
import yaml
from termcolor import colored
# from gptcache import cache
# from gptcache.adapter import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
import time
from experiments.analysis.utils import compute_f1_score_single_query
# cache.init()
# cache.set_openai_key()

In [5]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

def response_text(openai_resp):
    return openai_resp.choices[0].message.content

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completions_with_backoff(**kwargs):
    # time.sleep(2)
    return openai.ChatCompletion.create(**kwargs)

def replace_slot(text, entries):
    for key, value in entries.items():
        if not isinstance(value, str):
            value = str(value)
        new_text = text.replace("{{" + key +"}}", value.replace('"', "'"))
    return new_text
    
with open('/home/enhao/EQUI-VOCAL/inputs/id_to_query.json', 'r') as f:
    id_to_query = json.load(f)

In [3]:
demos_use_udfs = [
    {
      "role": "user",
      "content": "o0 is close to o1 while o0 is on the left."
    },
    {
      "role": "assistant",
      "content": "(Near(o0, o1, 1), LeftQuadrant(o0))"
    },
    {
      "role": "user",
      "content": "o0 and o1 are far apart, then they move close."
    },
    {
      "role": "assistant",
      "content": "Far(o0, o1, 3); Near(o0, o1, 1)"
    },
    {
      "role": "user",
      "content": "o0 is in front of o1 while o0 is on the left for at least 15 frames, then o0 moves to the right of o1 while o0 is at the top left for at least 5 frames."
    },
    {
      "role": "assistant",
      "content": "Duration((Frontof(o0, o1), LeftQuadrant(o0)), 15); Duration((LeftQuadrant(o0), RightOf(o0, o1), TopQuadrant(o0)), 5)"
    },
    {
      "role": "user",
      "content": "A sphere o0 is located to the left of a sphere o1 for at least one second, then o0 moves to the right of o1 but still to the left of o2, then o0 moves further to the right of o2 for at least 2 seconds."
    },
    {
      "role": "assistant",
      "content": "Duration((LeftOf(o0, o1), Shape(o0, 'sphere'), Shape(o1, 'sphere')), 25); (LeftOf(o0, o2), RightOf(o0, o1)); Duration(RightOf(o0, o2), 50)"
    },
    {
      "role": "user",
      "content": "A yellow object is at the bottom of the screen, then it moves to the top while another yellow object is at the bottom, then the second yellow object also moves to the top for at least a second."
    },
    {
      "role": "assistant",
      "content": "(BottomQuadrant(o0), Color(o0, 'yellow'), Color(o1, 'yellow')); (BottomQuadrant(o1), TopQuadrant(o0)); Duration(TopQuadrant(o1), 25)"
    },
]

demos_propose_udfs = [
    {
      "role": "user",
      "content": "o0 is close to o1 while o0 is on the left."
    },
    {
      "role": "assistant",
      "content": "(Near(o0, o1), Location_Left(o0))"
    },
    {
      "role": "user",
      "content": "o0 and o1 are far apart, then they move close."
    },
    {
      "role": "assistant",
      "content": "Far(o0, o1); Near(o0, o1)"
    },
    {
      "role": "user",
      "content": "o0 is in front of o1 while o0 is on the left for at least 15 frames, then o0 moves to the right of o1 while o0 is at the top left for at least 5 frames."
    },
    {
      "role": "assistant",
      "content": "Duration((Frontof(o0, o1), Location_Left(o0)), 15); Duration((Location_Left(o0), RightOf(o0, o1), Location_Top(o0)), 5)"
    },
    {
      "role": "user",
      "content": "A sphere o0 is located to the left of a sphere o1 for at least one second, then o0 moves to the right of o1 but still to the left of o2, then o0 moves further to the right of o2 for at least 2 seconds."
    },
    {
      "role": "assistant",
      "content": "Duration((LeftOf(o0, o1), Shape_Sphere(o0), Shape_Sphere(o1)), 25); (LeftOf(o0, o2), RightOf(o0, o1)); Duration(RightOf(o0, o2), 50)"
    },
    {
      "role": "user",
      "content": "A yellow object is at the bottom of the screen, then it moves to the top while another yellow object is at the bottom, then the second yellow object also moves to the top for at least a second."
    },
    {
      "role": "assistant",
      "content": "(Location_Bottom(o0), Color_Yellow(o0), Color_Yellow(o1)); (Location_Bottom(o1), Location_Top(o0)); Duration(Location_Top(o1), 25)"
    },
]

registered_functions = json.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/registered_udfs.json", "r"))

config = yaml.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/prompt.yaml", "r"), Loader=yaml.FullLoader)

system_prompt_use_udfs = config["system_prompt"]["use_udfs"]
system_prompt_parse_query = config["system_prompt"]["parse_query"]
system_prompt_verify_udfs = config["system_prompt"]["verify_udfs"]
system_prompt_implement_udfs = config["system_prompt"]["implement_udfs"]
prompt_propose_udfs = config["prompt"]["propose_udfs"]
prompt_verify_udfs = config["prompt"]["verify_udfs"]
demos_python_udf = json.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/python_udf.json", "r"))

In [4]:
def query_to_dsl(user_query, demos, prompt):
    messages = [{
      "role": "system",
      "content": prompt
    }]
    messages.extend(demos)
    messages.append({
      "role": "user",
      "content": replace_slot(config["prompt"]["parse_query"], {
        "user_query": user_query
        })
    })
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True, 
      messages=messages,
      temperature=0,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response, messages

In [67]:
# Clevrer queries
examples = [
    {
        "user_query": "Two objects move from far to close, then to far again",
        "gt_dsl": id_to_query["TQ2"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "o0 and o1 are far away, then they move close and o0 is behind o1",
        "gt_dsl": id_to_query["TQ3"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "o0 is in front of o1 while o0 is at the top",
        "gt_dsl": id_to_query["TQ5"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "Two objects move from close to far apart",
        "gt_dsl": id_to_query["TQ6"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "o0 is close to and behind o1 while o0 is on the left",
        "gt_dsl": id_to_query["TQ7"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "o0 at the bottom is far from o1, then they move close",
        "gt_dsl": id_to_query["TQ8"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "As o0 and o1 move from far to close, o0 is always on the left side of the scene",
        "gt_dsl": id_to_query["TQ9"],
        "dataset_name": "trajectories_handwritten"
    },
    {
        "user_query": "Two objects are far apart for at least 5 frames, then they move close, then they are far again",
        "gt_dsl": id_to_query["TQ1D"],
        "dataset_name": "trajectories_duration"
    },
    {
        "user_query": "o0 is on the left of o1 for at least 5 frames, then they move close to each other and o0 is at the top of the screen, then o0 is on the right of o1 for at least 5 frames",
        "gt_dsl": id_to_query["TQ2D"],
        "dataset_name": "trajectories_duration"
    },    
]
qids = ["TQ2", "TQ3", "TQ5", "TQ6", "TQ7", "TQ8", "TQ9", "TQ1D", "TQ2D"]
scores = [[] for qid in qids]

for _ in range(5):
    for idx, example in enumerate(examples):
        user_query = example["user_query"]
        response, _ = query_to_dsl(user_query, demos_use_udfs, system_prompt_use_udfs)
        parsed_dsl = response_text(response)
        gt_dsl = example["gt_dsl"]
        print("Total tokens in response:", response['usage']['total_tokens'])
        print("user_query: ", user_query)
        print("parsed_dsl: ", parsed_dsl)
        print("gt_dsl: ", gt_dsl)
        f1_score = compute_f1_score_single_query(parsed_dsl, gt_dsl, example["dataset_name"], 1, "/home/enhao/EQUI-VOCAL/inputs") 
        print("f1_score: ", f1_score)
        print()
        scores[idx].append(f1_score)

for idx, qid in enumerate(qids):
    print("{}: {}".format(qid, sum(scores[idx]) / len(scores[idx])))

Total tokens in response: 1199
user_query:  Two objects move from far to close, then to far again
parsed_dsl:  Far(o0, o1, 3); Near(o0, o1, 1); Far(o0, o1, 3)
gt_dsl:  Far(o0, o1, 3.0); Near(o0, o1, 1.0); Far(o0, o1, 3.0)
f1_score:  1.0

Total tokens in response: 1205
user_query:  o0 and o1 are far away, then they move close and o0 is behind o1
parsed_dsl:  Far(o0, o1, 3); (Near(o0, o1, 1), Behind(o0, o1))
gt_dsl:  Far(o0, o1, 3.0); (Behind(o0, o1), Near(o0, o1, 1.0))
f1_score:  1.0

Total tokens in response: 1188
user_query:  o0 is in front of o1 while o0 is at the top
parsed_dsl:  (FrontOf(o0, o1), TopQuadrant(o0))
gt_dsl:  (FrontOf(o0, o1), TopQuadrant(o0))
f1_score:  1.0

Total tokens in response: 1185
user_query:  Two objects move from close to far apart
parsed_dsl:  Near(o0, o1, 1); Far(o0, o1, 3)
gt_dsl:  Near(o0, o1, 1.0); Far(o0, o1, 3.0)
f1_score:  1.0

Total tokens in response: 1198
user_query:  o0 is close to and behind o1 while o0 is on the left
parsed_dsl:  (Near(o0, o1, 

* o1 and o2 move from far to close while o1 is on the left:
    *  ground-truth: (Far(o1, o2), Left(o1)); (Near(o1, o2), Left(o1))
    *  parsed: Far(o1, o2); (Near(o1, o2), Left(o1))
    *  user query is ambiguous

In [129]:
# User study queries
examples = [
    {
        "user_query": "A red object is far from a cylinder, then they get close",
        "gt_dsl": id_to_query["UQ1"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen",
        "gt_dsl": id_to_query["UQ2"]
    },
    {
        "user_query": "A red object is far from a cylinder, then they get close while a third object is at the top right of the screen.",
        "gt_dsl": id_to_query["UQ3"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen, then o1 moves to the top.",
        "gt_dsl": id_to_query["UQ4"]
    },
    {
        "user_query": "A red object is far from a cylinder for at least a second, then they are near each other while a third object is at the top right of the screen.",
        "gt_dsl": id_to_query["UQ5"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen, then o1 moves to the top, then a third object o2 is at the bottom right of the screen for at least a second.",
        "gt_dsl": id_to_query["UQ6"]
    }
]

qids = ["UQ1", "UQ2", "UQ3", "UQ4", "UQ5", "UQ6"]
scores = [[] for qid in qids]

for _ in range(5):
    for idx, example in enumerate(examples):
        try:
            user_query = example["user_query"]
            response, _ = query_to_dsl(user_query, demos_use_udfs, system_prompt_use_udfs)
            parsed_dsl = response_text(response)
            gt_dsl = example["gt_dsl"]
            print("Total tokens in response:", response['usage']['total_tokens'])
            print("user_query: ", user_query)
            print("parsed_dsl: ", parsed_dsl)
            print("gt_dsl: ", gt_dsl)
            f1_score = compute_f1_score_single_query(parsed_dsl, gt_dsl, "user_study_queries_scene_graph", 1, "/home/enhao/EQUI-VOCAL/inputs") 
            print("f1_score: ", f1_score)
            print()
            scores[idx].append(f1_score)
        except Exception as error:
            print("An exception occurred: {}. Set F1 score to 0.".format(error))
            print()
            scores[idx].append(0)

for idx, qid in enumerate(qids):
    print("{}: {}".format(qid, sum(scores[idx]) / len(scores[idx])))

Total tokens in response: 1112
user_query:  A red object is far from a cylinder, then they get close
parsed_dsl:  Far(o0, o1, 3), Color(o0, 'red'), Shape(o1, 'cylinder'); Near(o0, o1, 1)
gt_dsl:  (Color(o0, 'red'), Far(o0, o1, 3.0), Shape(o1, 'cylinder')); Near(o0, o1, 1.0)
1.0
f1_score:  1.0

Total tokens in response: 1112
user_query:  A purple metal object o0 is behind another object o1 at the bottom of the screen
parsed_dsl:  (Behind(o0, o1), BottomQuadrant(o1), Color(o0, 'purple'), Material(o0, 'metal'))
gt_dsl:  (Behind(o0, o1), BottomQuadrant(o1), Color(o0, 'purple'), Material(o0, 'metal'))
1.0
f1_score:  1.0

Total tokens in response: 1140
user_query:  A red object is far from a cylinder, then they get close while a third object is at the top right of the screen.
parsed_dsl:  Far(o0, o1, 3), Color(o0, 'red'), Shape(o1, 'cylinder'); (Near(o0, o1, 1), RightQuadrant(o2), TopQuadrant(o2))
gt_dsl:  (Color(o0, 'red'), Far(o0, o1, 3.0), Shape(o1, 'cylinder')); (Near(o0, o1, 1.0), Right

In [23]:
# Warsaw queries (should return empty string)
examples = [
    {
        "user_query": "o1 and o2 are close and in the same lane",
        "gt_dsl": "(LaneA(o1), LaneA(o2), Near(o1, o2))"
    },
    {
        "user_query": "In lane 1, o1 accelerates rapidly and then o2 accelerates rapdily",
        "gt_dsl": "(LaneA(o1), HighAccel(o1)); (LaneA(o2), HighAccel(o2))"
    },
    {
        "user_query": "o1 is turning from lane A into lane B while o2 is in lane B",
        "gt_dsl": "(LaneA(o1), LaneB(o2)); (LaneB(o1), LaneB(o2))"
    },
    {
        "user_query": "o1 merges from lane A into lane B while o2 is in lane C (which is next to lane B)",
        "gt_dsl": "(LaneA(o1), LaneC(o2)); (LaneB(o1), LaneC(o2))"
    },
    {
        "user_query": "o1 and o2 are in adjacent lanes and close for at least 5 frames",
        "gt_dsl": "Duration((LaneA(o1), LaneB(o2), Near(o1, o2)), 5)"
    },
    {
        "user_query": "o1 and o2 are in adjacent lanes and o1 is faster than o2 for at least 5 frames",
        "gt_dsl": "Duration((LaneA(o1), LaneB(o2), Faster(o1, o2)), 5)"
    },
]
for example in examples:
    user_query = example["user_query"]
    response, _ = query_to_dsl(user_query, demos_propose_udfs, system_prompt_parse_query)
    print("user_query: ", user_query)
    print("parsed_dsl: ", response_text(response))
    print("gt_dsl: ", example["gt_dsl"])
    print()

user_query:  o1 and o2 are close and in the same lane
parsed_dsl:  Input query can't be parsed using the existing functions.
gt_dsl:  (LaneA(o1), LaneA(o2), Near(o1, o2))

user_query:  In lane 1, o1 accelerates rapidly and then o2 accelerates rapdily
parsed_dsl:  Input query can't be parsed using the existing functions.
gt_dsl:  (LaneA(o1), HighAccel(o1)); (LaneA(o2), HighAccel(o2))

user_query:  o1 is turning from lane A into lane B while o2 is in lane B
parsed_dsl:  Input query can't be parsed using the existing functions.
gt_dsl:  (LaneA(o1), LaneB(o2)); (LaneB(o1), LaneB(o2))

user_query:  o1 merges from lane A into lane B while o2 is in lane C (which is next to lane B)
parsed_dsl:  Input query can't be parsed using the existing functions.
gt_dsl:  (LaneA(o1), LaneC(o2)); (LaneB(o1), LaneC(o2))

user_query:  o1 and o2 are in adjacent lanes and close for at least 5 frames
parsed_dsl:  Duration((Near(o1, o2)), 5)
gt_dsl:  Duration((LaneA(o1), LaneB(o2), Near(o1, o2)), 5)

user_query:

* Warsaw queries: 
    *  GPT-4 is able to identify missing predicates and therefore chooses to not parse query
    *  One exception: "o1 and o2 are in adjacent lanes and close for at least 5 frames"
    *  response formats are not consistent
    *  Some reasonings are very accurate, e.g.,
        *  "o1 and o2 are in adjacent lanes and o1 is faster than o2 for at least 5 frames": Sorry, your input can't be parsed as it contains predicates and conditions not covered in the existing options. The predicates "in adjacent lanes" and "faster than" are not available.

In [13]:
# Clevrer queries (variants)
examples = [
    {
        "user_query": "For a minimum of 5 frames, o0 is positioned to the left of o1, followed by a movement where they approach each other, with o0 eventually reaching the top of the screen. Subsequently, o0 shifts to the right of o1 for at least another 5 frames.",
        "gt_dsl": id_to_query["TQ2D"],
    },
    {
        "user_query": "Initially, o0 appears left of o1 for no less than 5 frames. They then converge, with o0 ascending to the screen's upper region. Following this, o0 is found to the right of o1, persisting in this position for a minimum of 5 frames.",
        "gt_dsl": id_to_query["TQ2D"],
    },
    {
        "user_query": "Starting with o0 to the left of o1 for 5 or more frames, they proceed to move towards one another, culminating with o0 at the screen's top. This is followed by o0 being positioned to the right of o1 for at least another 5 frames.",
        "gt_dsl": id_to_query["TQ2D"],
    },
    {
        "user_query": "o0 begins to the left of o1, maintaining this arrangement for at least 5 frames. They then draw nearer to each other, leading to o0 ascending to the top of the screen. Afterward, o0 is situated to the right of o1 for a minimum duration of 5 frames.",
        "gt_dsl": id_to_query["TQ2D"],
    },
    {
        "user_query": "Commencing with o0 on the left side of o1 for a duration of no less than 5 frames, they subsequently reduce their distance, resulting in o0 being at the top. This is followed by o0 relocating to the right side of o1 for at least 5 frames.",
        "gt_dsl": id_to_query["TQ2D"],
    },
    # {
    #     "user_query": "A red object is far from a cylinder for at least 1 minute, then they get close while a third object is at the top right of the screen",
    #     "gt_dsl": ""
    # },
]

qids = ["TQ2D variant1", "TQ2D variant2", "TQ2D variant3", "TQ2D variant4", "TQ2D variant5"]
scores = [[] for qid in qids]

for _ in range(5):
    for idx, example in enumerate(examples):
        try:
            user_query = example["user_query"]
            response, _ = query_to_dsl(user_query, demos_use_udfs, system_prompt_use_udfs)
            parsed_dsl = response_text(response)
            gt_dsl = example["gt_dsl"]
            print("Total tokens in response:", response['usage']['total_tokens'])
            print("user_query: ", user_query)
            print("parsed_dsl: ", parsed_dsl)
            print("gt_dsl: ", gt_dsl)
            f1_score = compute_f1_score_single_query(parsed_dsl, gt_dsl, "trajectories_duration", 1, "/home/enhao/EQUI-VOCAL/inputs") 
            print("f1_score: ", f1_score)
            print()
            scores[idx].append(f1_score)
        except Exception as error:
            print("An exception occurred: {}. Set F1 score to 0.".format(error))
            print()
            scores[idx].append(0)

for idx, qid in enumerate(qids):
    print("{}: {}".format(qid, sum(scores[idx]) / len(scores[idx])))

Total tokens in response: 1260
user_query:  For a minimum of 5 frames, o0 is positioned to the left of o1, followed by a movement where they approach each other, with o0 eventually reaching the top of the screen. Subsequently, o0 shifts to the right of o1 for at least another 5 frames.
parsed_dsl:  Duration((LeftOf(o0, o1)), 5); Near(o0, o1, 1); TopQuadrant(o0); Duration((RightOf(o0, o1)), 5)
gt_dsl:  Duration(LeftOf(o0, o1), 5); (Near(o0, o1, 1.0), TopQuadrant(o0)); Duration(RightOf(o0, o1), 5)
f1_score:  0.983435047951177

Total tokens in response: 1260
user_query:  Initially, o0 appears left of o1 for no less than 5 frames. They then converge, with o0 ascending to the screen's upper region. Following this, o0 is found to the right of o1, persisting in this position for a minimum of 5 frames.
parsed_dsl:  Duration((LeftOf(o0, o1)), 5); (Near(o0, o1, 1), TopQuadrant(o0)); Duration((RightOf(o0, o1)), 5)
gt_dsl:  Duration(LeftOf(o0, o1), 5); (Near(o0, o1, 1.0), TopQuadrant(o0)); Duratio

# UDF proposal

In [46]:
def query_to_dsl_with_udf_proposal(user_query, current_functions):
    # Step 1: parse user query into dsl
    response, messages = query_to_dsl(
        user_query, 
        demos_propose_udfs, 
        replace_slot(
            config["system_prompt"]["parse_query"], 
            {"functions": "\n".join(["{}: {}".format(key, value) for key, value in current_functions.items()])}
        )
    )
    print("user_query: ", user_query)
    print("[Step 1]\n", response_text(response))
    

    # Step 2.1: check if the query can't be parsed using the existing functions 
    if "Input query can't be parsed using the existing functions" in response_text(response):
        messages.append(response["choices"][0]["message"])
        messages.append(
            {
                "role": "user",
                "content": prompt_propose_udfs,
            }
        )
        # Step 2.2: propose new functions 
        response = completions_with_backoff(
          model="gpt-4-1106-preview",
          # model="gpt-4",
          # model="gpt-3.5-turbo", 
          # cache_skip=True,
          # tool_choice="none",  
          messages=messages,
          temperature=0,
          max_tokens=1024,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        print("[Step 2]\n ", response_text(response))
        proposed_functions = json.loads("\n\n".join(re.findall(r"```json\n(.*?)```", response_text(response), re.DOTALL)))
        # Step 3: verify functions (i.e., whether tehy can be constructed out of existing ones) 
        messages = [{
          "role": "system",
          "content": replace_slot(
                system_prompt_verify_udfs, 
                {"functions": "\n".join(["{}: {}".format(key, value) for key, value in current_functions.items()])}
            )
        }]
        messages.append({
          "role": "user",
          "content": replace_slot(prompt_verify_udfs, {
            "proposed_functions": "\n".join(["{}: {}".format(key, value) for key, value in proposed_functions.items()])
            })
        })
        response = completions_with_backoff(
          model="gpt-4-1106-preview",
          # model="gpt-4",
          # model="gpt-3.5-turbo", 
          # cache_skip=True, 
          messages=messages,
          temperature=0,
          max_tokens=1024,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        print("[Step 3]\n ", response_text(response))
        proposed_functions = json.loads("\n\n".join(re.findall(r"```json\n(.*?)```", response_text(response), re.DOTALL)))
        # Step 4: add functions and re-parse query 
        updated_functions = current_functions | proposed_functions
        # print("updated_functions:", updated_functions)
        response, messages = query_to_dsl(
            user_query, 
            demos_propose_udfs, 
            replace_slot(
                config["system_prompt"]["parse_query"], 
                {"functions": "\n".join(["{}: {}".format(key, value) for key, value in updated_functions.items()])}
            )
        )
        print("[Step 4]\n", response_text(response))
    # print("gt_dsl: ", example["gt_dsl"])
    print()
    return response_text(response)

In [39]:
# Clevrer queries (injecting unavailable predicates; propose new functions)
examples = [
    {
        "user_query": "Two objects are colliding with each other",
        "gt_dsl": ""
    },
    {
        "user_query": "Two objects are moving in parallel",
        "gt_dsl": ""
    },
    {
        "user_query": "A big, metal object o0 is behind another object o1 at the bottom of the screen",
        "gt_dsl": ""
    },
    {
        "user_query": "A metal object o0 is behind another fast-moving object o1 at the bottom of the screen",
        "gt_dsl": ""
    },
    {
        "user_query": "An orange object is far from a cylinder, then they get close while a third object is at the top right of the screen",
        "gt_dsl": ""
    },
    {
        "user_query": "A red object is far from a cone, then they get close while a third object is at the top right of the screen",
        "gt_dsl": ""
    },
]
for example in examples:
    print("***************************************************************************************************")
    user_query = example["user_query"]
    query_to_dsl_with_udf_proposal(user_query, registered_functions)

***************************************************************************************************
user_query:  Two objects are colliding with each other
[Step 1] parse dsl:  Input query can't be parsed using the existing functions.
[Step 2] propose udfs:
  Thought:
The user's query involves two objects that are colliding with each other. This implies a relationship predicate that captures the event of collision. Since the existing functions do not include any predicates that describe interactions or events such as a collision, we need to introduce a new relationship predicate that can represent this specific interaction between two objects.

The new function should not imply a change in state, but rather the event of collision itself. Therefore, we should not include predicates that describe the before and after states of a collision, such as "Approaching" or "SeparatedAfterCollision". Instead, we need a single predicate that captures the moment when two objects are in contact or col

In [42]:
# Warsaw queries; propose new functions
examples = [
    {
        "user_query": "o1 and o2 are close and in the same lane A",
        "gt_dsl": "(LaneA(o1), LaneA(o2), Near(o1, o2))"
    },
    {
        "user_query": "In lane A, o1 accelerates rapidly and then o2 accelerates rapdily",
        "gt_dsl": "(LaneA(o1), HighAccel(o1)); (LaneA(o2), HighAccel(o2))"
    },
    {
        "user_query": "While o2 is in lane B, o1 is turning from lane A into lane B",
        "gt_dsl": "(LaneA(o1), LaneB(o2)); (LaneB(o1), LaneB(o2))"
    },
    {
        "user_query": "While o2 is in lane C, o1 merges from lane A into lane B",
        "gt_dsl": "(LaneA(o1), LaneC(o2)); (LaneB(o1), LaneC(o2))"
    },
    {
        "user_query": "o1 is in lane A and o2 is in lane B and they are close for at least 5 frames",
        "gt_dsl": "Duration((LaneA(o1), LaneB(o2), Near(o1, o2)), 5)"
    },
    {
        "user_query": "o1 is in lane A and o2 is in lane B and o1 is faster than o2 for at least 5 frames",
        "gt_dsl": "Duration((LaneA(o1), LaneB(o2), Faster(o1, o2)), 5)"
    },
]
for example in examples:
    print("***************************************************************************************************")
    # Step 1: parse user query into dsl
    user_query = example["user_query"]
    query_to_dsl_with_udf_proposal(user_query, registered_functions)

***************************************************************************************************
user_query:  o1 and o2 are close and in the same lane A
[Step 1] parse dsl:  Input query can't be parsed using the existing functions.
[Step 2] propose udfs:
  Thought:
The user's query introduces a concept of objects being "in the same lane" which is not represented by any of the existing functions. The concept of a "lane" suggests a specific area or path within the frame that an object can occupy. Since the user specifies "lane A," we can infer that there may be multiple lanes, each potentially labeled with a different identifier. To address this, we need to introduce a function that can check whether an object is in a specific lane. We will not consider changes in states, so we only need a function that checks the current state of an object's lane.

Output:
```json
{
  "InLane_A(o0)": "Checks whether the object o0 is in lane A."
}
```
[Step 3] verified udfs:
  To determine if the prop

In [41]:
# User study queries with subset of functions
examples = [
    {
        "user_query": "A red object is far from a cylinder, then they get close",
        "gt_dsl": id_to_query["UQ1"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen",
        "gt_dsl": id_to_query["UQ2"]
    },
    {
        "user_query": "A red object is far from a cylinder, then they get close while a third object is at the top right of the screen.",
        "gt_dsl": id_to_query["UQ3"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen, then o1 moves to the top.",
        "gt_dsl": id_to_query["UQ4"]
    },
    {
        "user_query": "A red object is far from a cylinder for at least a second, then they are near each other while a third object is at the top right of the screen.",
        "gt_dsl": id_to_query["UQ5"]
    },
    {
        "user_query": "A purple metal object o0 is behind another object o1 at the bottom of the screen, then o1 moves to the top, then a third object o2 is at the bottom right of the screen for at least a second.",
        "gt_dsl": id_to_query["UQ6"]
    }
]

sampled_functions = {
    "Near(o0, o1)": "Whether o0 is near o1.",
    "Far(o0, o1)": "Whether o0 is far away from o1.",
    "LeftOf(o0, o1)": "Whether o0 is on the left of o1.",
    "RightOf(o0, o1)": "Whether o0 is on the right of o1.",
    "FrontOf(o0, o1)": "Whether o0 is in front of o1.",
    "Location_Left(o0)": "Whether o0 is on the left of the frame.",
    "Location_Top(o0)": "Whether o0 is at the top of the frame.",
    "Location_Bottom(o0)": "Whether o0 is at the bottom of the frame.",
    "Color_Yellow(o0)": "Whether the color of o0 is yellow.",
    "Shape_Sphere(o0)": "Whether the shape of o0 is sphere."
}

for example in examples:
    print("***************************************************************************************************")
    # Step 1: parse user query into dsl
    user_query = example["user_query"]
    query_to_dsl_with_udf_proposal(user_query, sampled_functions)
    print("gt_dsl", example["gt_dsl"])

***************************************************************************************************
user_query:  A red object is far from a cylinder, then they get close
[Step 1] parse dsl:  Input query can't be parsed using the existing functions.
[Step 2] propose udfs:
  Thought:
The user's query mentions a "red object" and a "cylinder," which are not covered by the existing functions. The existing functions allow us to check for the color yellow and the shape of a sphere, but not for the color red or the shape of a cylinder. Therefore, we need to introduce two new functions: one to check for the color red and another to check for the shape of a cylinder.

Output:
```json
{
  "Color_Red(o0)": "Whether the color of o0 is red",
  "Shape_Cylinder(o0)": "Whether the shape of o0 is a cylinder"
}
```
[Step 3] verified udfs:
  ```json
{
  "Color_Red(o0)": "Whether the color of o0 is red",
  "Shape_Cylinder(o0)": "Whether the shape of o0 is a cylinder"
}
```
[Step 3] parse dsl:  Far(o0, o1),

# Implement UDF

In [21]:
def implement_udf(proposed_function, demos, temperature=0, top_p=1):
    messages = [{
      "role": "system",
      "content": system_prompt_implement_udfs
    }]
    messages.extend(demos_python_udf)
    messages.append(
        {
            "role": "user",
            "content": proposed_function,
        }
    )
    # pretty_print_conversation(messages)
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=temperature,
      max_tokens=1024,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    print(response_text(response))
    function_implementation = "\n\n".join(re.findall(r"```sql\n(.*?)```", response_text(response), re.DOTALL))
    print()
    return function_implementation

In [9]:
proposed_functions = [
    "Far(o0, o1): Whether o0 is far away from o1.",
    "LeftOf(o0, o1): Whether o0 is on the left of o1.",
    "RightOf(o0, o1): Whether o0 is on the right of o1.",
    "Behind(o0, o1): Whether o0 is behind o1.",
    "FrontOf(o0, o1): Whether o0 is in front of o1.",
    "Location_Left(o0): Whether o0 is on the left of the frame.",
    "Location_Right(o0): Whether o0 is on the right of the frame.",
    "Location_Top(o0): Whether o0 is at the top of the frame.",
    "Location_Bottom(o0): Whether o0 is at the bottom of the frame.",
    "Color_Gray(o0): Whether the color of o0 is gray.",
    "Color_Red(o0): Whether the color of o0 is red.",
    "Color_Blue(o0): Whether the color of o0 is blue.",
    "Color_Green(o0): Whether the color of o0 is green.",
    "Color_Brown(o0): Whether the color of o0 is brown.",
    "Color_Cyan(o0): Whether the color of o0 is cyan.",
    "Color_Purple(o0): Whether the color of o0 is purple.",
    "Color_Yellow(o0): Whether the color of o0 is yellow.",
    "Shape_Cube(o0): Whether the shape of o0 is cube.",
    "Shape_Sphere(o0): Whether the shape of o0 is sphere.",
    "Shape_Cylinder(o0): Whether the shape of o0 is cylinder.",
    "Material_Rubber(o0): Whether the material of o0 is rubber.",
    "Material_Metal(o0): Whether the material of o0 is metal."
]
for proposed_function in proposed_functions:
    print("***************************************************************************************************")
    print("[python udf]\n", implement_udf(proposed_function, demos_python_udf))

***************************************************************************************************
To determine if one object (`o0`) is behind another object (`o1`), we need to compare their positions along the x-axis. If the front edge of `o0` is further along the x-axis than the back edge of `o1`, then `o0` can be considered to be behind `o1`. In a 2D coordinate system with the origin at the top-left corner, this means that `o0`'s `x1` should be greater than `o1`'s `x2`.

Here's how you can create a PostgreSQL function using Python to determine if `o0` is behind `o1`:

```sql
CREATE OR REPLACE FUNCTION pybehind(o0_x1 float, o0_y1 float, o0_x2 float, o0_y2 float, o1_x1 float, o1_y1 float, o1_x2 float, o1_y2 float)
RETURNS boolean
AS $$
    # Check if o0's front edge is further along the x-axis than o1's back edge
    return o0_x1 > o1_x2
$$ LANGUAGE plpython3u;
```

This function assumes that the objects are rectangles aligned with the axes, and it only checks the x-axis to determine

## Function refinement with user feedback

In [24]:
# Generate 10 function implementations, with a temperature of 1.0 (more createive and random)

proposed_function = "Far(o0, o1): Whether o0 is far away from o1."
for _ in range(10):
    print("***************************************************************************************************")
    print("[python udf]\n", implement_udf(proposed_function, demos_python_udf, temperature=1))

***************************************************************************************************
```sql
CREATE OR REPLACE FUNCTION pyfar (o1_x1 float, o1_y1 float, o1_x2 float, o1_y2 float, o2_x1 float, o2_y1 float, o2_x2 float, o2_y2 float)
    RETURNS boolean
AS $$
    import math

    def calculate_distance(x1, y1, x2, y2):
        """Calculate the Euclidean distance between two points."""
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

    # Calculate the center points of the bounding boxes for both objects
    center_o1 = ((o1_x1 + o1_x2) / 2.0, (o1_y1 + o1_y2) / 2.0)
    center_o2 = ((o2_x1 + o2_x2) / 2.0, (o2_y1 + o2_y2) / 2.0)

    # Calculate the distance between the center points
    distance = calculate_distance(*center_o1, *center_o2)

    # Define a threshold for being "far"
    # Assuming "far" means not directly adjacent or close
    # For this, we use a threshold of half the width of the frame, 480 / 2 = 240, and half the height of the frame, 320 / 2 = 160

In [31]:
# Refine function implementation with labeled examples
# Far relationship, with definition the distance between the center points of two objects 
# is at least 3 times the average width of the objects. 
proposed_function = "Far(o0, o1): Whether o0 is far away from o1."
examples = """
Examples: 
o0 = ('cylinder', 'purple', 'metal', 356, 73, 396, 117),
o1 = ('cube', 'green', 'metal', 0, 9, 1, 17),
o2 = ('cylinder', 'purple', 'metal', 226, 41, 256, 81),
o3 = ('cube', 'red', 'rubber', 360, 172, 432, 252),
o4 = ('cylinder', 'gray', 'rubber', 145, 4, 170, 32),
o5 = ('cylinder', 'green', 'metal', 335, 67, 375, 110),
o6 = ('cylinder', 'purple', 'metal', 236, 80, 274, 127),
o7 = ('cylinder', 'gray', 'rubber', 59, 52, 99, 92),
o8 = ('cylinder', 'yellow', 'rubber', 280, 40, 312, 78),
o9 = ('sphere', 'purple', 'rubber', 148, 59, 181, 92),
o10 = ('sphere', 'blue', 'rubber', 192, 98, 231, 137),
o11 = ('cube', 'blue', 'metal', 228, 39, 267, 79),
o12 = ('cylinder', 'yellow', 'rubber', 227, 134, 273, 194),
o13 = ('cylinder', 'blue', 'metal', 126, 113, 171, 170),
o14 = ('sphere', 'brown', 'rubber', 344, 131, 390, 175),
o15 = ('cylinder', 'cyan', 'metal', 365, 105, 410, 159),
o16 = ('sphere', 'cyan', 'rubber', 40, 144, 88, 190),
o17 = ('cylinder', 'brown', 'metal', 144, 94, 185, 144),
o18 = ('cylinder', 'red', 'rubber', 156, 111, 200, 166),
o19 = ('cylinder', 'purple', 'rubber', 123, 38, 156, 76), 
Far(o0, o1) = True,
Far(o2, o3) = True,
Far(o4, o5) = True,
Far(o6, o7) = True,
Far(o8, o9) = True,
Far(o10, o11) = False,
Far(o12, o13) = False,
Far(o14, o15) = False,
Far(o16, o17) = False,
Far(o18, o19) = False
"""

for _ in range(10):
    print("***************************************************************************************************")
    messages = [{
      "role": "system",
      "content": config["system_prompt"]["implement_udfs_with_input_output_examples"]
    }]
    messages.extend(json.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/python_udf_with_input_output_examples.json", "r")))
    messages.append(
        {
            "role": "user",
            "content": proposed_function + examples,
        }
    )
    # pretty_print_conversation(messages)
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=0,
      max_tokens=1024,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    print(response_text(response))
    function_implementation = "\n\n".join(re.findall(r"```sql\n(.*?)```", response_text(response), re.DOTALL))
    print("[python udf]\n", function_implementation)

***************************************************************************************************
To create a PostgreSQL function that determines whether two objects are far from each other, we can use a similar approach to the `Near` function but with a different threshold. Here's how you can define the `pyfar` function:

```sql
CREATE OR REPLACE FUNCTION pyfar (o1_x1 float, o1_y1 float, o1_x2 float, o1_y2 float, o2_x1 float, o2_y1 float, o2_x2 float, o2_y2 float)
    RETURNS boolean
AS $$
    import math
    # Calculate the center points of both objects
    cx1 = (o1_x1 + o1_x2) / 2
    cy1 = (o1_y1 + o1_y2) / 2
    cx2 = (o2_x1 + o2_x2) / 2
    cy2 = (o2_y1 + o2_y2) / 2

    # Calculate the distance between the center points
    distance = math.sqrt(math.pow(cx1 - cx2, 2.0) + math.pow(cy1 - cy2, 2.0))

    # Define a threshold for being "far"
    # The threshold is set based on the examples provided, assuming that "far" means a distance greater than the average size of the objects

In [30]:
# Refine function implementation with labeled examples
# Far relationship, with definition the distance between the center points of two objects 
# is at least 5 times the average width of the objects. 
proposed_function = "Far(o0, o1): Whether o0 is far away from o1."
examples = """
Examples: 
o0 = ('cylinder', 'brown', 'rubber', 378, 110, 426, 164),
o1 = ('cylinder', 'green', 'rubber', 64, 180, 125, 252),
o2 = ('sphere', 'red', 'metal', 60, 109, 100, 148),
o3 = ('cylinder', 'purple', 'metal', 271, 129, 317, 189),
o4 = ('cylinder', 'yellow', 'metal', 134, 1, 158, 25),
o5 = ('cube', 'yellow', 'metal', 363, 95, 423, 150),
o6 = ('cylinder', 'blue', 'rubber', 180, 33, 209, 68),
o7 = ('cylinder', 'purple', 'metal', 332, 151, 386, 216),
o8 = ('sphere', 'yellow', 'metal', 1, 130, 22, 171),
o9 = ('sphere', 'purple', 'rubber', 294, 75, 331, 110),
o10 = ('cube', 'yellow', 'rubber', 339, 142, 406, 208),
o11 = ('sphere', 'brown', 'rubber', 120, 114, 161, 154),
o12 = ('sphere', 'gray', 'metal', 201, 52, 232, 83),
o13 = ('cube', 'brown', 'metal', 109, 67, 157, 112),
o14 = ('sphere', 'gray', 'rubber', 24, 210, 84, 267),
o15 = ('sphere', 'blue', 'rubber', 57, 175, 109, 226),
o16 = ('cube', 'purple', 'rubber', 193, 130, 244, 192),
o17 = ('cylinder', 'green', 'metal', 15, 61, 56, 101),
o18 = ('cube', 'red', 'metal', 251, 38, 285, 76),
o19 = ('cylinder', 'gray', 'metal', 296, 39, 327, 77),
Far(o0, o1) = True,
Far(o2, o3) = True,
Far(o4, o5) = True,
Far(o6, o7) = True,
Far(o8, o9) = True,
Far(o10, o11) = False,
Far(o12, o13) = False,
Far(o14, o15) = False,
Far(o16, o17) = False,
Far(o18, o19) = False
"""
for _ in range(10):
    print("***************************************************************************************************")
    messages = [{
      "role": "system",
      "content": config["system_prompt"]["implement_udfs_with_input_output_examples"]
    }]
    messages.extend(json.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/python_udf_with_input_output_examples.json", "r")))
    messages.append(
        {
            "role": "user",
            "content": proposed_function + examples,
        }
    )
    # pretty_print_conversation(messages)
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=1,
      max_tokens=1024,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    print(response_text(response))
    function_implementation = "\n\n".join(re.findall(r"```sql\n(.*?)```", response_text(response), re.DOTALL))
    print("[python udf]\n", function_implementation)

***************************************************************************************************
```sql
CREATE OR REPLACE FUNCTION pyfar (o1_x1 float, o1_y1 float, o1_x2 float, o1_y2 float, o2_x1 float, o2_y1 float, o2_x2 float, o2_y2 float)
    RETURNS boolean
AS $$
    import math
    # Calculate the center points of both objects
    cx1 = (o1_x1 + o1_x2) / 2
    cy1 = (o1_y1 + o1_y2) / 2
    cx2 = (o2_x1 + o2_x2) / 2
    cy2 = (o2_y1 + o2_y2) / 2

    # Calculate the distance between the center points
    distance = math.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)

    # Define a threshold for being "far"
    frame_diagonal = math.sqrt(480 ** 2 + 320 ** 2)
    threshold = frame_diagonal * 0.5 # Change the threshold accordingly based on the desired definition of "far"

    # Return whether the objects are far from each other
    return distance > threshold
$$ LANGUAGE plpython3u;
```

Please adjust the threshold value `0.5` factor of the `frame_diagonal` as needed to properly repres

In [40]:
# Refine function implementation with labeled examples
# Far relationship, with definition the distance between the center points of two objects 
# is at least 5 times the average width of the objects. 
proposed_function = "Far(o0, o1): Whether o0 is far away from o1."
examples = """
Examples: 
o0 = ('cylinder', 'brown', 'rubber', 378, 110, 426, 164),
o1 = ('cylinder', 'green', 'rubber', 64, 180, 125, 252),
o2 = ('sphere', 'red', 'metal', 60, 109, 100, 148),
o3 = ('cylinder', 'purple', 'metal', 271, 129, 317, 189),
o4 = ('cylinder', 'yellow', 'metal', 134, 1, 158, 25),
o5 = ('cube', 'yellow', 'metal', 363, 95, 423, 150),
o6 = ('cylinder', 'blue', 'rubber', 180, 33, 209, 68),
o7 = ('cylinder', 'purple', 'metal', 332, 151, 386, 216),
o8 = ('sphere', 'yellow', 'metal', 1, 130, 22, 171),
o9 = ('sphere', 'purple', 'rubber', 294, 75, 331, 110),
o10 = ('cube', 'yellow', 'rubber', 339, 142, 406, 208),
o11 = ('sphere', 'brown', 'rubber', 120, 114, 161, 154),
o12 = ('sphere', 'gray', 'metal', 201, 52, 232, 83),
o13 = ('cube', 'brown', 'metal', 109, 67, 157, 112),
o14 = ('sphere', 'gray', 'rubber', 24, 210, 84, 267),
o15 = ('sphere', 'blue', 'rubber', 57, 175, 109, 226),
o16 = ('cube', 'purple', 'rubber', 193, 130, 244, 192),
o17 = ('cylinder', 'green', 'metal', 15, 61, 56, 101),
o18 = ('cube', 'red', 'metal', 251, 38, 285, 76),
o19 = ('cylinder', 'gray', 'metal', 296, 39, 327, 77),
Far(o0, o1) = True,
Far(o2, o3) = True,
Far(o4, o5) = True,
Far(o6, o7) = True,
Far(o8, o9) = True,
Far(o10, o11) = False,
Far(o12, o13) = False,
Far(o14, o15) = False,
Far(o16, o17) = False,
Far(o18, o19) = False
"""
for _ in range(1):
    print("***************************************************************************************************")
    messages = [{
      "role": "system",
      "content": config["system_prompt"]["implement_udfs_with_input_output_examples"]
    }]
    messages.extend(json.load(open("/home/enhao/EQUI-VOCAL/experiments/gpt/python_udf_with_input_output_examples.json", "r")))
    messages.append(
        {
            "role": "user",
            "content": proposed_function + examples,
        }
    )
    # pretty_print_conversation(messages)
    # response = completions_with_backoff(
    #   model="gpt-4-1106-preview",
    #   # model="gpt-4",
    #   # model="gpt-3.5-turbo", 
    #   # cache_skip=True,
    #   # tool_choice="none",  
    #   messages=messages,
    #   temperature=0,
    #   max_tokens=1024,
    #   top_p=1,
    #   frequency_penalty=0,
    #   presence_penalty=0
    # )
    # print(response_text(response))
    messages.append(
        {
            "role": "assistant",
            "content": """
            ```sql
            CREATE OR REPLACE FUNCTION pyfar (o1_x1 float, o1_y1 float, o1_x2 float, o1_y2 float, o2_x1 float, o2_y1 float, o2_x2 float, o2_y2 float)
                RETURNS boolean
            AS $$
                import math
                # Calculate the center points of both objects
                cx1 = (o1_x1 + o1_x2) / 2
                cy1 = (o1_y1 + o1_y2) / 2
                cx2 = (o2_x1 + o2_x2) / 2
                cy2 = (o2_y1 + o2_y2) / 2
            
                # Calculate the euclidean distance between the center points
                distance = math.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)
            
                # Far threshold - this may need adjustment based on provided examples
                # Note: There's no definitive method provided in the question for judging far objects.
                # A heuristic is used here which assumes objects further than half the max distance of the frame are considered far.
                frame_max_distance = math.sqrt(480**2 + 320**2) / 2
            
                # Return whether the objects are far from each other
                return distance > frame_max_distance
            $$ LANGUAGE plpython3u;
            ```
            """,
        }
    )
    function_implementation = "\n\n".join(re.findall(r"```sql\n(.*?)```", response_text(response), re.DOTALL))
    # print("[python udf]\n", function_implementation)
    # Step 2
    messages.append(
        {
            "role": "user",
            "content": "The function above fails the given unit test:\n assert pyfar(60, 109, 100, 148, 271, 129, 317, 189) == True\n assert pyfar(134, 1, 158, 25, 363, 95, 423, 150) == True\n assert pyfar(180, 33, 209, 68, 332, 151, 386, 216) == True\n Trace the execution of the function for each unit test, and then fix the function",
        }
    )
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=0,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    messages.append(response['choices'][0]['message'])
    # Step 3
    messages.append(
        {
            "role": "user",
            "content": "The function above fails the given unit test:\n assert pyfar(339, 142, 406, 208, 120, 114, 161, 154) == False\n assert pyfar(201, 52, 232, 83, 109, 67, 157, 112) == False\n assert pyfar(193, 130, 244, 192, 15, 61, 56, 101) == False\n Trace the execution of the function for each unit test, and then fix the function",
        }
    )
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=0,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    messages.append(response['choices'][0]['message'])
    pretty_print_conversation(messages)
    

***************************************************************************************************
system: Please implement a PostgreSQL function in Python procedural language based on the description and the input/output examples using only standard python libraries. The implemented function must be as consistent as possible with the input/output examples. An object o has the following attributes: (shape varchar, color varchar, material varchar, x1 float, y1 float, x2 float, y2 float), where x1, y1, x2, y2 are the upper-left and bottom-right coordinates of the bounding box. Coordinates of the bounding box's corners are calculated with respect to the top-left corner of the image which has (x, y) coordinates (0, 0). Assume the frame size is 480x320. Prefix The PostgreSQL function name with "py". Inputs to the PostgreSQL function consist of a subset of attribues of o and the function returns a boolean value. The function must follow the format: ```sql ```


user: Near(o0, o1): Whether o

In [4]:
for _ in range(1):
    print("***************************************************************************************************")
    messages = [
        {
            "role": "system",
            "content": "You are an expert programming assistant.",
        }
    ]
    messages.append(
        {
          "role": "user",
          "content": "Complete the following task in Python. \nDetermine whether object o0 is far away from object o1.\n    An object is represented by a dictionary of {shape: string, color: string, material: string, x1: float, y1: float, x2: float, y2: float}, where x1, y1, x2, y2 are the upper-left and bottom-right coordinates of the bounding box. Coordinates of the bounding box's corners are calculated with respect to the top-left corner of the image which has (x, y) coordinates (0, 0). Assume the frame size is 480x320.\n    Input:\n        o1: an object, represented as a dictionary of {shape: string, color: string, material: string, x1: float, y1: float, x2: float, y2: float}\n        o2: an object, represented as a dictionary of {shape: string, color: string, material: string, x1: float, y1: float, x2: float, y2: float}\n        **kwargs: additional parameters that can be adjusted as needed\n    Output: \n        result: a boolean value \n\ndef pyfar(o1, o2, **kwargs): "
        }
    )
    response = completions_with_backoff(
      model="gpt-4-1106-preview",
      # model="gpt-4",
      # model="gpt-3.5-turbo", 
      # cache_skip=True,
      # tool_choice="none",  
      messages=messages,
      temperature=0.7,
      max_tokens=1024,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    print(response_text(response))

***************************************************************************************************


RetryError: RetryError[<Future at 0x7f126c583730 state=finished raised APIRemovedInV1>]